In [1]:
import time
import gym
import numpy as np
from gym import spaces
from tqdm import tqdm


import numpy as np
from utils.skeleton import *
from utils.quaternion import *
from utils.blazepose import blazepose_skeletons
import os
from pypot.creatures import PoppyTorso
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#from pypot.creatures.ik import IKChain
from pypot.primitive.move import Move
from pypot.primitive.move import MovePlayer

import sys
sys.path.append('E:\Anaconda\envs\gym-examples')
import gym_examples
from gym.wrappers import FlattenObservation

In [2]:
env = gym.make('gym_examples/Poppy-v0')
env.reset()

pygame 2.1.0 (SDL 2.0.16, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
Hello, I am Poppy!


array([ 0.10876056, -0.17632997,  0.07149935, -0.10220377, -0.18020962,
        0.07149935], dtype=float32)

In [3]:
# from stable_baselines3.common.env_checker import check_env
# check_env(env)

In [ ]:
# import gymnasium as gym
# import numpy as np

from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

#env = gym.make("Pendulum-v1")

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1)
model.learn(total_timesteps=10000, log_interval=10)
model.save("ddpg_pendulum")
vec_env = model.get_env()

del model # remove to demonstrate saving and loading

model = DDPG.load("ddpg_pendulum")

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    env.render("human")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
current step :  0
reward :  -0.5384182625583659
current step :  1
reward :  -0.5956370267118066
current step :  2
reward :  -0.6055087440824913
current step :  3
reward :  -0.5280508418959943
current step :  4
reward :  -0.5935060657773511
current step :  5
reward :  -0.4942909123090172
current step :  6
reward :  -0.5307782781205065
current step :  7
reward :  -0.5669936555440136
current step :  8
reward :  -0.38575322786227717
current step :  9
reward :  -0.22591723389166712
current step :  10
reward :  -0.5617109177713228
current step :  11
reward :  -0.37579662866682517
current step :  12
reward :  -0.36740543357216054
current step :  13
reward :  -0.47998385693500994
current step :  14
reward :  -0.388490587909829
current step :  15
reward :  -0.6337576925649188
current step :  16
reward :  -0.32024231589037844
current step :  17
reward :  -0.3128493274212501
current step :  18
reward :  

current step :  164
reward :  -0.5191265486747183
current step :  165
reward :  -0.5351969258475934
current step :  166
reward :  -0.48896060659171364
current step :  167
reward :  -0.5870780735170852
current step :  168
reward :  -0.5522709359252458
current step :  169
reward :  -0.64107173353893
current step :  170
reward :  -0.6150283108038324
current step :  171
reward :  -0.6177506778166646
current step :  172
reward :  -0.5505308477085339
current step :  173
reward :  -0.5239530148950814
current step :  174
reward :  -0.594169982266194
current step :  175
reward :  -0.5036942704858018
current step :  176
reward :  -0.44707860713942715
current step :  177
reward :  -0.5951255762833401
current step :  178
reward :  -0.481606568544016
current step :  179
reward :  -0.6325805126018185
current step :  180
reward :  -0.4961922037764499
current step :  181
reward :  -0.5987723966159575
current step :  182
reward :  -0.5835837437066049
current step :  183
reward :  -0.35670690305272124
c

current step :  43
reward :  -0.46621235464959593
current step :  44
reward :  -0.4669297877116466
current step :  45
reward :  -0.4435601584327577
current step :  46
reward :  -0.42893489337145846
current step :  47
reward :  -0.44397903856558085
current step :  48
reward :  -0.43940141763069507
current step :  49
reward :  -0.4271109872622637
current step :  50
reward :  -0.46231791270934314
current step :  51
reward :  -0.424278359606463
current step :  52
reward :  -0.46500161130021156
current step :  53
reward :  -0.44958625687937026
current step :  54
reward :  -0.4383499676343283
current step :  55
reward :  -0.4511319304198915
current step :  56
reward :  -0.450926395053497
current step :  57
reward :  -0.4658974220848745
current step :  58
reward :  -0.4639441240071953
current step :  59
reward :  -0.4645592983737351
current step :  60
reward :  -0.4457537764765414
current step :  61
reward :  -0.39735944545380897
current step :  62
reward :  -0.45538212377149906
current step 

current step :  207
reward :  -0.4025044952214036
current step :  208
reward :  -0.4257760117678088
current step :  209
reward :  -0.40821584862553467
current step :  210
reward :  -0.4282496334360473
current step :  211
reward :  -0.42339272548354606
current step :  212
reward :  -0.4403609252951209
current step :  213
reward :  -0.414389582887539
current step :  214
reward :  -0.3973582745786539
current step :  215
reward :  -0.4202082783812957
current step :  216
reward :  -0.3527627652557288
current step :  217
reward :  -0.41167188047877684
current step :  218
reward :  -0.38132356228555775
current step :  219
reward :  -0.4208106421370223
current step :  220
reward :  -0.39927914286884375
current step :  221
reward :  -0.3640301799940062
current step :  222
reward :  -0.4205755117529198
current step :  223
reward :  -0.3784377199516408
current step :  224
reward :  -0.34885262617637386
current step :  225
reward :  -0.4258466165027219
current step :  226
reward :  -0.415973982119

In [ ]:
torch.cuda.is_available()